<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

# **Launch Sites Locations Analysis with Folium**

Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import folium
import wget
import pandas as pd
#import json

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


# Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [3]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

Now, you can take a look at what are the coordinates for each site.


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


<div class="alert alert-warning" role="alert">
  <b>Note</b>: <br> There are a number of errors with the <b>launch_sites_df</b> dataset. The next section will correct these errors.
</div>

### Correcting launch site errors

These launch sites have the follonwing issues:

1. Duplicated launch sites. **CCAFS LC-40** and **CCAFS SLC-40** are actually the same launch site. The original name of this site is Cape Canaveral Air Force Station Launch Complex 40 (CCAFS LC-40). The name was updated to Cape Canaveral Air Force Station Space Launch Complex (CCAFS SLC-40) when SpaceX started using the site. Therefore; CCAFS LC-40 is redundant and needs to be renamed. According to [Google Maps](https://www.google.ca/maps/place/Space+Launch+Complex+40/@28.5636458,-80.5828757,3629m/data=!3m1!1e3!4m14!1m7!3m6!1s0x88e0bb299cde9295:0x917e8e53b6d3d1a1!2sSpace+Launch+Complex+40!8m2!3d28.5621041!4d-80.5772921!16zL20vMGdfY2oy!3m5!1s0x88e0bb299cde9295:0x917e8e53b6d3d1a1!8m2!3d28.5621041!4d-80.5772921!16zL20vMGdfY2oy?entry=ttu&g_ep=EgoyMDI1MTExMi4wIKXMDSoASAFQAw%3D%3D), the correct Lat / Long for this site is: `28.562311086191027, -80.57732773608846`. These values will be updated as well.

2. The correct Lat / Long for Kennedy Space Center Launch Complex 39A (KSC LC-39A) is: `28.608488832338782, -80.60393030935165`, according to [Google Maps](https://www.google.ca/maps/place/Launch+Pad+39A/@28.608325,-80.6072772,1701m/data=!3m1!1e3!4m10!1m2!2m1!1sKSC+LC-39A!3m6!1s0x88e0bbcacef2d2cd:0x25eb79aabef8827d!8m2!3d28.608325!4d-80.6039429!15sCgpLU0MgTEMtMzlBWgwiCmtzYyBsYyAzOWGSARFhZXJvc3BhY2VfY29tcGFueZoBI0NoWkRTVWhOTUc5blMwVkpRMEZuU1VOYU1rdEljbGxSRUFFqgFDEAEqDiIKa3NjIGxjIDM5YSgAMh8QASIbCOWZ06wwdbbNgTBJZh-7mdi7VjI9QbKfTbJ_Mg4QAiIKa3NjIGxjIDM5YeABAPoBBAgAEEg!16s%2Fg%2F11f38v94zk?entry=ttu&g_ep=EgoyMDI1MTExMi4wIKXMDSoASAFQAw%3D%3D).

3. The correct Lat / Long for Vandenberg Air Force Base (VAFB SLC-4E) is: `34.63217046029318, -120.6106023879881`, according to [Google Maps](https://www.google.ca/maps/place/Vandenberg+AFB+Space+Launch+Complex+4/@34.6320264,-120.61324,1970m/data=!3m2!1e3!4b1!4m6!3m5!1s0x80ec23f11fbb6627:0xb2890f36a4a066a3!8m2!3d34.632022!4d-120.6106597!16s%2Fm%2F04jdq7d?entry=ttu&g_ep=EgoyMDI1MTExMi4wIKXMDSoASAFQAw%3D%3D).

In [5]:
# Update Lat / Long at index #'s: 0 and 1
lat_long_replacement_dct = {
    0 : {
        # CCAFS SLC-40 Coordinates (from Google Maps)
        'Lat' : 28.562311086191027,   
        'Long' : -80.57732773608846,
        'Site Name' : 'Cape Canaveral Air Force Station Space Launch Complex 40'
    },
    1 : {
        'Lat' : 28.608488832338782,
        'Long' : -80.60393030935165,
        'Site Name' : 'Kennedy Space Center Launch Complex 39A'
    },
    2 : {
        'Lat' : 34.63217046029318,
        'Long' : -120.6106023879881,
        'Site Name' : 'Vandenberg Air Force Base Space Launch Complex 4E'
    }   
}

In [6]:
# Drop CCAFS LC-40 from record
launch_sites_df = launch_sites_df.drop(index = 0).reset_index(drop = True)

# Make the required updates to the Lat / Long values in the dataset
for idx, values in lat_long_replacement_dct.items():
    launch_sites_df.loc[idx, values.keys()] = values.values()

# Verify output
launch_sites_df

,Launch Site,Lat,Long,Site Name
0,CCAFS SLC-40,28.562311,-80.577328,Cape Canaveral Air Force Station Space Launch ...
1,KSC LC-39A,28.608489,-80.603930,Kennedy Space Center Launch Complex 39A
2,VAFB SLC-4E,34.632170,-120.610602,Vandenberg Air Force Base Space Launch Complex 4E


<div class="alert alert-info" role="alert">
  The next step is to update the data found in the <b>spacex_df</b> dataset to maintain consistency.
</div>

### Correcting SpaceX dataset errors

In [7]:
# Rename the launch site
spacex_df['Launch Site'] = spacex_df['Launch Site'].replace(
    'CCAFS LC-40', 
    'CCAFS SLC-40'
)

# Verify operation
spacex_df['Launch Site'].unique()

array(['CCAFS SLC-40', 'VAFB SLC-4E', 'KSC LC-39A'], dtype=object)

In [8]:
# Build a map of launch site values to update the spacex_df dataset
launch_map = launch_sites_df.set_index('Launch Site')

# Verify operation
launch_map

,Lat,Long,Site Name
Launch Site,,,
CCAFS SLC-40,28.562311,-80.577328,Cape Canaveral Air Force Station Space Launch ...
KSC LC-39A,28.608489,-80.603930,Kennedy Space Center Launch Complex 39A
VAFB SLC-4E,34.632170,-120.610602,Vandenberg Air Force Base Space Launch Complex 4E


In [9]:
# Update the Lat and Long coordinates in spacex_df
spacex_df['Lat'] = spacex_df['Launch Site'].map(launch_map['Lat'])
spacex_df['Long'] = spacex_df['Launch Site'].map(launch_map['Long'])
spacex_df['Site Name'] = spacex_df['Launch Site'].map(launch_map['Site Name'])

# Verify operations
print(spacex_df['Lat'].unique())
print(spacex_df['Long'].unique())
print(spacex_df['Site Name'].unique())

[28.56231109 34.63217046 28.60848883]
[ -80.57732774 -120.61060239  -80.60393031]
['Cape Canaveral Air Force Station Space Launch Complex 40'
 'Vandenberg Air Force Base Space Launch Complex 4E'
 'Kennedy Space Center Launch Complex 39A']


### Creating the map

Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be [NASA Mission Control Center (Johnson Space Center) at Houston, Texas](https://www.google.ca/maps/place/NASA+Mission+Control+Center/@29.559257,-95.0957139,4166m/data=!3m1!1e3!4m9!1m2!2m1!1sNASA+Johnson+Space+Center!3m5!1s0x86409da671292593:0xf684f098a7237a30!8m2!3d29.5580992!4d-95.0893337!16s%2Fg%2F11b7c2zvpk?entry=ttu&g_ep=EgoyMDI1MTExMi4wIKXMDSoASAFQAw%3D%3D).


In [10]:
# Start location is NASA Mission Control Center
nasa_coordinate = [29.558464964859915, -95.08804613590078] # Coordinates updated using Google Maps 

# Create a map object
site_map = folium.Map(
    location = nasa_coordinate, 
    zoom_start = 10
)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [11]:
# Create a circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(
    nasa_coordinate, 
    radius = 1000, 
    color = '#d35400', 
    fill = True
).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a yellow circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size = (20,20),
        icon_anchor = (0,0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [12]:
# Reset the site_map object
site_map = None

In [13]:
# Initialize a new map object
site_map = folium.Map(
    location = nasa_coordinate, 
    zoom_start = 5
)

In [14]:
# Helper function to plot the names of each launch site on a map
def launch_site_marker(launch_point):
    # Displays in the HTML Tag
    location_name = launch_sites_df.iloc[launch_point, 0]
    # Displays in the marker name
    site_description = launch_sites_df.iloc[launch_point, 3]
    
    # Create a circle
    circle = folium.Circle(location = [launch_sites_df.iloc[launch_point, 1], launch_sites_df.iloc[launch_point, 2]],
                           radius = 50,
                           color = '#d35400', 
                           fill = True
                          ).add_child(folium.Popup(site_description))

    # Create a marker to describe the circle
    marker = folium.map.Marker(
    location = [launch_sites_df.iloc[launch_point, 1], launch_sites_df.iloc[launch_point, 2]],
    # Create an icon as a text label
    icon = DivIcon(
         icon_size = (20, 20),
         icon_anchor = (0, 0),
         html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' %location_name
         )
    )
    
    return site_map.add_child(circle), site_map.add_child(marker)

In [15]:
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for launch_point in range(len(launch_sites_df)):
    launch_site_marker(launch_point)

In [16]:
# Display map
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [17]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,Site Name
46,KSC LC-39A,28.608489,-80.603930,1,Kennedy Space Center Launch Complex 39A
47,KSC LC-39A,28.608489,-80.603930,1,Kennedy Space Center Launch Complex 39A
48,KSC LC-39A,28.608489,-80.603930,1,Kennedy Space Center Launch Complex 39A
49,CCAFS SLC-40,28.562311,-80.577328,1,Cape Canaveral Air Force Station Space Launch ...
50,CCAFS SLC-40,28.562311,-80.577328,1,Cape Canaveral Air Force Station Space Launch ...
51,CCAFS SLC-40,28.562311,-80.577328,0,Cape Canaveral Air Force Station Space Launch ...
52,CCAFS SLC-40,28.562311,-80.577328,0,Cape Canaveral Air Force Station Space Launch ...
53,CCAFS SLC-40,28.562311,-80.577328,0,Cape Canaveral Air Force Station Space Launch ...
54,CCAFS SLC-40,28.562311,-80.577328,1,Cape Canaveral Air Force Station Space Launch ...
55,CCAFS SLC-40,28.562311,-80.577328,0,Cape Canaveral Air Force Station Space Launch ...


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [18]:
marker_cluster = MarkerCluster()

_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [19]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [20]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,Site Name,marker_color
46,KSC LC-39A,28.608489,-80.603930,1,Kennedy Space Center Launch Complex 39A,green
47,KSC LC-39A,28.608489,-80.603930,1,Kennedy Space Center Launch Complex 39A,green
48,KSC LC-39A,28.608489,-80.603930,1,Kennedy Space Center Launch Complex 39A,green
49,CCAFS SLC-40,28.562311,-80.577328,1,Cape Canaveral Air Force Station Space Launch ...,green
50,CCAFS SLC-40,28.562311,-80.577328,1,Cape Canaveral Air Force Station Space Launch ...,green
51,CCAFS SLC-40,28.562311,-80.577328,0,Cape Canaveral Air Force Station Space Launch ...,red
52,CCAFS SLC-40,28.562311,-80.577328,0,Cape Canaveral Air Force Station Space Launch ...,red
53,CCAFS SLC-40,28.562311,-80.577328,0,Cape Canaveral Air Force Station Space Launch ...,red
54,CCAFS SLC-40,28.562311,-80.577328,1,Cape Canaveral Air Force Station Space Launch ...,green
55,CCAFS SLC-40,28.562311,-80.577328,0,Cape Canaveral Air Force Station Space Launch ...,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [21]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was a success or failed
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # Create and add a Marker cluster to the site map
    marker = folium.Marker(
        [record['Lat'], record['Long']], 
        icon = folium.Icon(
            color = 'white', 
            icon_color = record['marker_color']
        )
    )
    marker_cluster.add_child(marker)

# Display map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [22]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position = 'topright',
    separator = ' Long: ',
    empty_string = 'NaN',
    lng_first = False,
    num_digits = 20,
    prefix = 'Lat:',
    lat_formatter = formatter,
    lng_formatter = formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [23]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [24]:
# find coordinate of the closet coastline (CCAFS SLC-40)
# e.g.,: Lat: 28.562311086191027  Lon: -80.57732773608846
# Lat / Long of CCAFS SLC-40
launch_site_lat = launch_sites_df.loc[0, 'Lat']
launch_site_lon = launch_sites_df.loc[0, 'Long']

# Lat / Long of coastline
coastline_lat = 28.56287
coastline_lon = -80.56766

# Calculate distance to the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [25]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
   [coastline_lat, coastline_lon],
   icon = DivIcon(
       icon_size = (20,20),
       icon_anchor = (0,0),
       html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )

site_map.add_child(distance_marker)

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [26]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines = folium.PolyLine(
    locations = [
        [
            launch_site_lat,
            launch_site_lon
        ],
        [
            coastline_lat,
            coastline_lon
        ]
    ], 
    weight = 1
)

site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [27]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

## CCAFS SLC-40 Closest city, railway, highway, coastline

### Create the base map of the CCAFS SLC-40 area

In [28]:
# Start location is SLC-40 launch complex
ccafs_slc_40_coordinate = [
    launch_sites_df.loc[0, 'Lat'], 
    launch_sites_df.loc[0, 'Long']
] 

location_name = launch_sites_df.loc[0, 'Launch Site']

# Create a map object
ccafs_slc_40_site_map = folium.Map(
    location = ccafs_slc_40_coordinate, 
    zoom_start = 10
)

# Path to geojson file of SLC-40
geojson_path = "../data/00_Datasets/02_GIS/CCAFS SLC-40.geojson"

# Add GeoJSON overlay to the map
folium.GeoJson(
    geojson_path,
    name = "SLC-40 Boundary",
    style_function = lambda x : {
        "fillColor" : "blue",
        "color" : "darkblue",
        "weight" : 2,
        "fillOpacity" : 0.2,
    }
).add_to(ccafs_slc_40_site_map).add_child(folium.Popup(location_name));

In [29]:
# Generate map
ccafs_slc_40_site_map

### Create a dictionary of key locations (Coastline, Railway, Highway, City) close to SLC-40

In [30]:
# Dictionary of key locations around SLC-40
slc_40_key_locations_dct = {
    0 : {
        'Name' : 'Coast Line',
        'Description' : 'The Atlantic Ocean',
        'Marker Color' : 'blue',
        'Icon' : 'umbrella-beach',
        'Lat' : 28.562833602280854,
        'Long' : -80.56792153462233
    },
    1 : {
        'Name' : 'Railway',
        'Description' : 'NASA Jay-Jay Rail Yard',
        'Marker Color' : 'black',
        'Icon' : 'train',
        'Lat' : 28.650799438732392,  
        'Long' : -80.81511328591485
    },
    2 : {
        'Name' : 'Highway',
        'Description' : 'U.S. Route 1',
        'Marker Color' : 'gray',
        'Icon' : 'road',
        'Lat' : 28.561240960294178,
        'Long' : -80.79884941923731
    },
    3 : {
        'Name' : 'City',
        'Description' : 'Titusville',
        'Marker Color' : 'red',
        'Icon' : 'city',
        'Lat' : 28.610013726027997,
        'Long' : -80.80926761486283
    }
}

### Generate a map

In [31]:
# Add key locations to the SLC-40 map
for k, loc in slc_40_key_locations_dct.items():
    # Create variables to store values
    loc_lat = loc['Lat']
    loc_lon = loc['Long']
    loc_name = loc['Name']
    loc_icon = loc['Icon']
    loc_desc = loc['Description']
    loc_color = loc['Marker Color']

    # Create the marker
    popup_html = f"<b>{loc_name}</b><br>{loc_desc}"
    folium.Marker(
        [loc_lat, loc_lon],
        popup = popup_html,
        icon = folium.Icon(color = loc_color, icon = loc_icon, prefix = 'fa')
    ).add_to(ccafs_slc_40_site_map)

    # Calculate distance
    distance_km = calculate_distance(ccafs_slc_40_coordinate[0], ccafs_slc_40_coordinate[1], loc_lat, loc_lon)
    distance_label = f"{distance_km:.2f} km"

    # Draw lines from SLC-40 to the locations
    line = folium.PolyLine(
        locations = [(ccafs_slc_40_coordinate[0], ccafs_slc_40_coordinate[1]), (loc_lat, loc_lon)],
        color = loc_color,
        weight = 2
    )

    # Add distance label as popup on the line
    line.add_child(folium.Popup(f"<b>Distance to {loc_name}:</b> {distance_label}"))
    line.add_to(ccafs_slc_40_site_map)

    # Print the distances
    print(f'The distance from SLC-40 to the closest {loc_name} ({loc_desc}) is {distance_label}')


# Display the map
ccafs_slc_40_site_map

The distance from SLC-40 to the closest Coast Line (The Atlantic Ocean) is 0.92 km
The distance from SLC-40 to the closest Railway (NASA Jay-Jay Rail Yard) is 25.22 km
The distance from SLC-40 to the closest Highway (U.S. Route 1) is 21.64 km
The distance from SLC-40 to the closest City (Titusville) is 23.27 km


## KSC LC-39A Closest city, railway, highway, coastline

### Create the base map of the KSC LC-39A area

In [32]:
# Start location is SLC-40 launch complex
ksc_lc_39a_coordinate = [
    launch_sites_df.loc[1, 'Lat'], 
    launch_sites_df.loc[1, 'Long']
] 

location_name = launch_sites_df.loc[1, 'Launch Site']

# Create a map object
ksc_lc_39a_site_map = folium.Map(
    location = ksc_lc_39a_coordinate, 
    zoom_start = 10
)

# Path to geojson file of LC-39A
geojson_path = "../data/00_Datasets/02_GIS/KSC LC-39A.geojson"

# Add GeoJSON overlay to the map
folium.GeoJson(
    geojson_path,
    name = "LC-39A Boundary",
    style_function = lambda x : {
        "fillColor" : "blue",
        "color" : "darkblue",
        "weight" : 2,
        "fillOpacity" : 0.2,
    }
).add_to(ksc_lc_39a_site_map).add_child(folium.Popup(location_name));

In [33]:
# Generate map
ksc_lc_39a_site_map

### Create a dictionary of key locations (Coastline, Railway, Highway, City) close to LC-39A

In [34]:
# Dictionary of key locations around LC-39A
# These locations are the same as SLC-40, with the exception of the coast line
lc_39a_key_locations_dct = {
    0 : {
        'Name' : 'Coast Line',
        'Description' : 'The Atlantic Ocean',
        'Marker Color' : 'blue',
        'Icon' : 'umbrella-beach',
        'Lat' : 28.609018819266403, 
        'Long' : -80.59453162830434
    },
    1 : {
        'Name' : 'Railway',
        'Description' : 'NASA Jay-Jay Rail Yard',
        'Marker Color' : 'black',
        'Icon' : 'train',
        'Lat' : 28.650799438732392,  
        'Long' : -80.81511328591485
    },
    2 : {
        'Name' : 'Highway',
        'Description' : 'U.S. Route 1',
        'Marker Color' : 'gray',
        'Icon' : 'road',
        'Lat' : 28.561240960294178,
        'Long' : -80.79884941923731
    },
    3 : {
        'Name' : 'City',
        'Description' : 'Titusville',
        'Marker Color' : 'red',
        'Icon' : 'city',
        'Lat' : 28.610013726027997,
        'Long' : -80.80926761486283
    }
}

### Generate a map

In [35]:
# Add key locations to the LC-39A map
for k, loc in lc_39a_key_locations_dct.items():
    # Create variables to store values
    loc_lat = loc['Lat']
    loc_lon = loc['Long']
    loc_name = loc['Name']
    loc_icon = loc['Icon']
    loc_desc = loc['Description']
    loc_color = loc['Marker Color']

    # Create the marker
    popup_html = f"<b>{loc_name}</b><br>{loc_desc}"
    folium.Marker(
        [loc_lat, loc_lon],
        popup = popup_html,
        icon = folium.Icon(color = loc_color, icon = loc_icon, prefix = 'fa')
    ).add_to(ksc_lc_39a_site_map)

    # Calculate distance
    distance_km = calculate_distance(ksc_lc_39a_coordinate[0], ksc_lc_39a_coordinate[1], loc_lat, loc_lon)
    distance_label = f"{distance_km:.2f} km"

    # Draw lines from LC-39A to the locations
    line = folium.PolyLine(
        locations = [(ksc_lc_39a_coordinate[0], ksc_lc_39a_coordinate[1]), (loc_lat, loc_lon)],
        color = loc_color,
        weight = 2
    )

    # Add distance label as popup on the line
    line.add_child(folium.Popup(f"<b>Distance to {loc_name}:</b> {distance_label}"))
    line.add_to(ksc_lc_39a_site_map)

    # Print the distances
    print(f'The distance from LC-39A to the closest {loc_name} ({loc_desc}) is {distance_label}')


# Display the map
ksc_lc_39a_site_map

The distance from LC-39A to the closest Coast Line (The Atlantic Ocean) is 0.92 km
The distance from LC-39A to the closest Railway (NASA Jay-Jay Rail Yard) is 21.15 km
The distance from LC-39A to the closest Highway (U.S. Route 1) is 19.75 km
The distance from LC-39A to the closest City (Titusville) is 20.05 km


## VAFB SLC-4E Closest city, railway, highway, coastline

In [36]:
launch_sites_df

,Launch Site,Lat,Long,Site Name
0,CCAFS SLC-40,28.562311,-80.577328,Cape Canaveral Air Force Station Space Launch ...
1,KSC LC-39A,28.608489,-80.603930,Kennedy Space Center Launch Complex 39A
2,VAFB SLC-4E,34.632170,-120.610602,Vandenberg Air Force Base Space Launch Complex 4E


### Create the base map of the VAFB SLC-4E area

In [37]:
# Start location is SLC-4E launch complex
vafb_slc_4e_coordinate = [
    launch_sites_df.loc[2, 'Lat'], 
    launch_sites_df.loc[2, 'Long']
] 

location_name = launch_sites_df.loc[2, 'Launch Site']

# Create a map object
vafb_slc_4e_site_map = folium.Map(
    location = vafb_slc_4e_coordinate, 
    zoom_start = 10
)

# Path to geojson file of LC-39A
geojson_path = "../data/00_Datasets/02_GIS/VAFB SLC-4E.geojson"

# Add GeoJSON overlay to the map
folium.GeoJson(
    geojson_path,
    name = "SLC-4E Boundary",
    style_function = lambda x : {
        "fillColor" : "blue",
        "color" : "darkblue",
        "weight" : 2,
        "fillOpacity" : 0.2,
    }
).add_to(vafb_slc_4e_site_map).add_child(folium.Popup(location_name));

In [38]:
# Generate map
vafb_slc_4e_site_map

### Create a dictionary of key locations (Coastline, Railway, Highway, City) close to SLC-4E 

In [39]:
# Dictionary of key locations around SLC-4E
slc_4e_key_locations_dct = {
    0 : {
        'Name' : 'Coast Line',
        'Description' : 'The Pacific Ocean',
        'Marker Color' : 'blue',
        'Icon' : 'umbrella-beach',
        'Lat' : 34.63281568762727, 
        'Long' : -120.62662922843366
    },
    1 : {
        'Name' : 'Railway',
        'Description' : 'Lompoc-Surf Station',
        'Marker Color' : 'black',
        'Icon' : 'train',
        'Lat' : 34.68299644692548, 
        'Long' : -120.60477777198383
    },
    2 : {
        'Name' : 'Highway',
        'Description' : 'Highway 1 (Pacific Coast Highway)',
        'Marker Color' : 'gray',
        'Icon' : 'road',
        'Lat' : 34.63235232647258, 
        'Long' : -120.43203801081728
    },
    3 : {
        'Name' : 'City',
        'Description' : 'Lompoc',
        'Marker Color' : 'red',
        'Icon' : 'city',
        'Lat' : 34.639749905687324,
        'Long' : -120.46487711112127
    }
}

### Generate a map

In [40]:
# Add key locations to the SLC-4E map
for k, loc in slc_4e_key_locations_dct.items():
    # Create variables to store values
    loc_lat = loc['Lat']
    loc_lon = loc['Long']
    loc_name = loc['Name']
    loc_icon = loc['Icon']
    loc_desc = loc['Description']
    loc_color = loc['Marker Color']

    # Create the marker
    popup_html = f"<b>{loc_name}</b><br>{loc_desc}"
    folium.Marker(
        [loc_lat, loc_lon],
        popup = popup_html,
        icon = folium.Icon(color = loc_color, icon = loc_icon, prefix = 'fa')
    ).add_to(vafb_slc_4e_site_map)

    # Calculate distance
    distance_km = calculate_distance(vafb_slc_4e_coordinate[0], vafb_slc_4e_coordinate[1], loc_lat, loc_lon)
    distance_label = f"{distance_km:.2f} km"

    # Draw lines from LC-39A to the locations
    line = folium.PolyLine(
        locations = [(vafb_slc_4e_coordinate[0], vafb_slc_4e_coordinate[1]), (loc_lat, loc_lon)],
        color = loc_color,
        weight = 2
    )

    # Add distance label as popup on the line
    line.add_child(folium.Popup(f"<b>Distance to {loc_name}:</b> {distance_label}"))
    line.add_to(vafb_slc_4e_site_map)

    # Print the distances
    print(f'The distance from SLC-4E to the closest {loc_name} ({loc_desc}) is {distance_label}')


# Display the map
vafb_slc_4e_site_map

The distance from SLC-4E to the closest Coast Line (The Pacific Ocean) is 1.47 km
The distance from SLC-4E to the closest Railway (Lompoc-Surf Station) is 5.68 km
The distance from SLC-4E to the closest Highway (Highway 1 (Pacific Coast Highway)) is 16.34 km
The distance from SLC-4E to the closest City (Lompoc) is 13.36 km


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
